In [1]:
from SmartApi.smartConnect import SmartConnect
import pyotp
import pandas as pd
import requests
import pandas as pd
from datetime import datetime
from datetime import datetime, timedelta
import pandas_ta as ta


api_key = 'C46AImmE'
clientId = 'M507690'
pwd = '4663'
smartApi = SmartConnect(api_key)
token = "4ACNNSIZL6EV6UQ2ACSI23H5VQ"
totp=pyotp.TOTP(token).now()
correlation_id = "abc123"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

#fetch User Profile
userProfile= smartApi.getProfile(refreshToken)
User= pd.DataFrame(userProfile)
User = User['data']['name']
User
print(f"Welcome {User} to the World of SMART API BY ANGEL.....!!")
print(f"Login Sucessfull....!!")


[I 251221 01:08:25 smartConnect:124] in pool


Welcome MANA  BHATTACHARYYA to the World of SMART API BY ANGEL.....!!
Login Sucessfull....!!


In [2]:
# ---- NIFTY 50 Token (Angel One Fixed Token) ----
NIFTY_TOKEN = "26000"      # NIFTY 50 Index
EXCHANGE = "NSE"

# ---- Fetch LTP ----
ltp_data = smartApi.ltpData(
    exchange=EXCHANGE,
    tradingsymbol="NIFTY",
    symboltoken=NIFTY_TOKEN
)

print("📈 NIFTY 50 LTP:", ltp_data['data']['ltp'])

ATM = round(ltp_data['data']['ltp'] / 50) * 50
print("📈 NIFTY 50 SPOT Price:", ATM)

ltp =  ltp_data['data']['ltp']
prev_close = ltp_data['data']['close']

change = ltp - prev_close
pct_change = (change / prev_close) * 100

print(f"NIFTY Change: {change:.2f}")
print(f"NIFTY % Change: {pct_change:.2f}%")

📈 NIFTY 50 LTP: 25966.4
📈 NIFTY 50 SPOT Price: 25950
NIFTY Change: 150.85
NIFTY % Change: 0.58%


In [3]:
url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
instruments = pd.DataFrame(requests.get(url).json())

expiry = instruments[(instruments['name']=='NIFTY') &  (instruments['instrumenttype']=='OPTIDX') &  (instruments['exch_seg']=='NFO')]["expiry"].unique()
expiry

today = datetime.today()

dates = pd.to_datetime(expiry, format="%d%b%Y")
next_expiry = dates[dates > today].min()
print("Next Expiry:", next_expiry.strftime("%d%b%y").upper())

Next Expiry: 23DEC25


In [4]:
expiry_next = next_expiry.strftime("%d%b%y").upper()

def get_option_symbol(expiry_next, strike, opt_type):
    return f"NIFTY{expiry_next}{strike}{opt_type}"

ce_symbol = get_option_symbol(expiry_next, ATM, "CE")
pe_symbol = get_option_symbol(expiry_next, ATM, "PE")

print("CE Symbol:", ce_symbol)
print("PE Symbol:", pe_symbol)

CE Symbol: NIFTY23DEC2525950CE
PE Symbol: NIFTY23DEC2525950PE


In [5]:
def get_option_token(symbol):
    row = instruments[instruments["symbol"] == symbol]
    if row.empty:
        raise Exception(f"Token not found for {symbol}")
    return row.iloc[0]["token"]

ce_token = get_option_token(ce_symbol)
pe_token = get_option_token(pe_symbol)

print("CE Token:", ce_token)
print("PE Token:", pe_token)

CE Token: 57003
PE Token: 57004


In [6]:
def fetch_5min_candles(token):
    to_date = datetime.now()
    from_date = to_date - timedelta(days=3)

    response = smartApi.getCandleData({
        "exchange": "NFO",
        "symboltoken": token,
        "interval": "THREE_MINUTE",
        "fromdate": from_date.strftime("%Y-%m-%d %H:%M"),
        "todate": to_date.strftime("%Y-%m-%d %H:%M")
    })

    if not response or "data" not in response or response["data"] is None:
        raise Exception("No candle data")

    df = pd.DataFrame(
        response["data"],
        columns=["datetime", "open", "high", "low", "close", "volume"]
    )

    # 🔑 CRITICAL FIX
    df["datetime"] = pd.to_datetime(df["datetime"])
    df.set_index("datetime", inplace=True)
    df.sort_index(inplace=True)

    return df



def apply_indicators(df):
    df = df.copy()

    df["EMA14"] = ta.ema(df["close"], length=5)
    df["EMA20"] = ta.ema(df["close"], length=10)
    df["RSI"] = ta.rsi(df["close"], length=14)

    df["VWAP"] = ta.vwap(
        high=df["high"],
        low=df["low"],
        close=df["close"],
        volume=df["volume"]
    )

    df.dropna(inplace=True)
    return df

pe_df = fetch_5min_candles(ce_token)
pe_df = fetch_5min_candles(pe_token)

pe_df = apply_indicators(pe_df)
pe_df = apply_indicators(pe_df)

In [7]:
pe_df.reset_index()
pe_df = pe_df.to_csv('PE_Data.csv')
pe_df = pd.read_csv('PE_Data.csv')
latest = pe_df.tail(3)
latest

,datetime,open,high,low,close,volume,EMA14,EMA20,RSI,VWAP
229,2025-12-19 15:21:00+05:30,63.45,64.45,63.3,63.55,24986,63.603538,64.072345,43.036636,78.406643
230,2025-12-19 15:24:00+05:30,63.60,64.50,62.7,63.70,42055,63.635692,64.004646,43.522379,78.286303
231,2025-12-19 15:27:00+05:30,63.55,63.95,60.0,60.30,59607,62.523795,63.331074,36.023888,78.093757


In [8]:
df = pe_df
df['target_price'] = df['close'].shift(-1)
df['target_pct'] = (df['close'].shift(-1) - df['close']) / df['close']
df['ema_diff'] = df['EMA14'] - df['EMA20']
df['price_vwap_diff'] = df['close'] - df['VWAP']
df['rsi_signal'] = df['RSI'] - 50

df['hl_range'] = df['high'] - df['low']
df['oc_range'] = df['close'] - df['open']

df.dropna(inplace=True)



from sklearn.model_selection import TimeSeriesSplit

features = [
    'open','high','low','close','volume',
    'EMA14','EMA20','RSI','VWAP',
    'ema_diff','price_vwap_diff','rsi_signal',
    'hl_range','oc_range'
]

X = df[features]
y = df['target_price']   # or target_pct




In [9]:
from xgboost import XGBRegressor

model = XGBRegressor(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror'
)

model.fit(X[:-1], y[:-1])

print("Model trained successfully")

Model trained successfully


In [10]:
latest = X.iloc[-1].values.reshape(1, -1)
predicted_price = model.predict(latest)[0]

print("Current Close :", df['close'].iloc[-1])
print("Predicted Next Close :", round(predicted_price, 2))


Current Close : 63.7
Predicted Next Close : 62.88


In [11]:
if predicted_price > df['close'].iloc[-1]:
    signal = "BUY 📈"
else:
    signal = "SELL 📉"
print("Signal:", signal)



from sklearn.metrics import mean_absolute_error

preds = model.predict(X[:-1])
mae = mean_absolute_error(y[:-1], preds)
print("MAE:", round(mae, 2))



Signal: SELL 📉
MAE: 0.19


In [12]:
features = [
    'open','high','low','close','volume',
    'EMA14','EMA20','RSI','VWAP',
    'ema_diff','price_vwap_diff','rsi_signal',
    'hl_range','oc_range'
]

df['future_price'] = model.predict(df[features])




df['signal'] = 0  # 1 = BUY, -1 = SELL

df.loc[
    (df['future_price'] > df['close']) &
    (df['close'] > df['VWAP']) &
    (df['RSI'] > 50),
    'signal'
] = 1

df.loc[
    (df['future_price'] < df['close']) &
    (df['close'] < df['VWAP']) &
    (df['RSI'] < 50),
    'signal'
] = -1



df['next_close'] = df['close'].shift(-1)

df['result'] = 0
df.loc[(df['signal'] == 1) & (df['next_close'] > df['close']), 'result'] = 1
df.loc[(df['signal'] == -1) & (df['next_close'] < df['close']), 'result'] = 1

total_trades = df['signal'].abs().sum()
successful_trades = df['result'].sum()

success_rate = (successful_trades / total_trades) * 100 if total_trades > 0 else 0


df['actual_direction'] = (df['next_close'] > df['close']).astype(int)
df['pred_direction'] = (df['future_price'] > df['close']).astype(int)

directional_accuracy = (
    (df['actual_direction'] == df['pred_direction']).mean() * 100
)

model_features = features.copy()
df['future_price'] = model.predict(df[model_features])



In [13]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df = df.tail(60)
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=("Price + AI Prediction + Signals", "RSI")
)

# --- PRICE LINES ---
fig.add_trace(go.Scatter(
    x=df['datetime'], y=df['close'],
    name='Close', line=dict(color='white')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df['datetime'], y=df['VWAP'],
    name='VWAP', line=dict(color='orange', dash='dot')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df['datetime'], y=df['EMA14'],
    name='EMA14', line=dict(color='cyan')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df['datetime'], y=df['future_price'],
    name='AI Future Price', line=dict(color='magenta', dash='dash')
), row=1, col=1)

# --- BUY SIGNALS ---
fig.add_trace(go.Scatter(
    x=df[df['signal'] == 1]['datetime'],
    y=df[df['signal'] == 1]['close'],
    mode='markers',
    name='BUY',
    marker=dict(color='green', size=10, symbol='triangle-up')
), row=1, col=1)

# --- SELL SIGNALS ---
fig.add_trace(go.Scatter(
    x=df[df['signal'] == -1]['datetime'],
    y=df[df['signal'] == -1]['close'],
    mode='markers',
    name='SELL',
    marker=dict(color='red', size=10, symbol='triangle-down')
), row=1, col=1)

# --- RSI ---
fig.add_trace(go.Scatter(
    x=df['datetime'], y=df['RSI'],
    name='RSI', line=dict(color='yellow')
), row=2, col=1)

fig.add_hline(y=70, line_dash="dash", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", row=2, col=1)

fig.update_layout(
    height=900,
    template='plotly_dark',
    title=f"""
AI Trading Dashboard
Backtest Success Rate: {success_rate:.2f}%
Future Direction Accuracy: {directional_accuracy:.2f}%
""",
    xaxis_rangeslider_visible=False
)

fig.show()


In [14]:
print("Total Trades:", int(total_trades))
print("Successful Trades:", int(successful_trades))
print("Backtest Success Rate:", round(success_rate, 2), "%")
print("Future Direction Accuracy:", round(directional_accuracy, 2), "%")

Total Trades: 104
Successful Trades: 102
Backtest Success Rate: 98.08 %
Future Direction Accuracy: 99.57 %
